## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

from IPython.display import HTML

# import sys
# sys.path.append('../../')

from openbb_terminal.reports import widget_helpers as widgets
from openbb_terminal.sdk import openbb
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal.core.config.paths import REPOSITORY_DIRECTORY

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

pd.set_option("display.float_format", lambda x: "%.5f" % x)

In [ ]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = widgets.html_report_stylesheet()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
symbol = "BTC"
report_name = f"Crypto Report for {symbol}"

In [ ]:
ticker = symbol

In [ ]:
author = ""
report_title = f"INVESTMENT RESEARCH REPORT ON {ticker.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
# basic info

basic_info = openbb.crypto.dd.basic(ticker)
if not basic_info.empty:
    basic_info = basic_info.set_index("Metric")
    basic_info

In [ ]:
news = openbb.crypto.dd.news(symbol=ticker).head(10)

if (isinstance(news, pd.DataFrame) and not news.empty) or (
    not isinstance(news, pd.DataFrame) and news
):
    news = news.set_index("published_at")
    news.sort_index()
    news["link"] = news["link"].apply(lambda x: f'<a href="{x}">{x}</a>')
    news
else:
    news = pd.DataFrame()

In [ ]:
links = openbb.crypto.dd.links(ticker)

if (isinstance(links, pd.DataFrame) and not links.empty) or (
    not isinstance(links, pd.DataFrame) and links
):
    links["Link"] = links["Link"].apply(lambda x: f'<a href="{x}">{x}</a>')
    links
else:
    links = pd.DataFrame()

In [ ]:
five_year = datetime.datetime.now() - datetime.timedelta(days=1825)
three_year = datetime.datetime.now() - datetime.timedelta(days=1095)
one_year = datetime.datetime.now() - datetime.timedelta(days=365)
three_months = datetime.datetime.now() - datetime.timedelta(days=90)

dates = {
    "three_months": three_months,
    "one_year": one_year,
    "three_year": three_year,
    "five_year": five_year,
}

In [ ]:
def get_historical_data(ticker, dict_of_dates, is_plot=True):
    list_of_historical_data = []

    for date in dict_of_dates:
        data = openbb.crypto.load(
            symbol=ticker,
            to_symbol="usd",
            start_date=dict_of_dates[date],
            source="YahooFinance",
        )

        if data.empty:
            continue

        # we'll be using the adjusted close price as same as close price
        data["Adj Close"] = data["Close"]

        if is_plot:
            fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
            plt.suptitle(
                f"{date} historical data for {ticker.upper()}",
                fontsize=14,
                color="black",
            )
            data.plot(y="Close", ax=ax)
            plt.ylabel("USDT")

            fig.tight_layout()
            f = io.BytesIO()
            fig.savefig(f, format="svg")
            plot_data = f.getvalue().decode("utf-8")

            list_of_historical_data.append(plot_data)
        else:
            list_of_historical_data.append(data)

    return list_of_historical_data

In [ ]:
try:
    historical_data = get_historical_data(ticker, dates)
except Exception as e:
    raise KeyboardInterrupt

In [ ]:
historical_data_one_year = get_historical_data(ticker, {"one_year": one_year}, False)[0]
historical_data_one_year

In [ ]:
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import show_quick_performance

show_quick_performance(
    crypto_df=historical_data_one_year,
    symbol=ticker,
    current_currency="USDT",
    source="YahooFinance",
    exchange="binance",
    interval="1440",
)

fig = plt.figure(figsize=(11, 3), dpi=150)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
quick_performance = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.crypto.dd.headlines_chart(ticker, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
headlines = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.crypto.chart(
    historical_data_one_year,
    to_symbol=ticker,
    from_symbol="USDT",
    source="YahooFinance",
    exchange="binance",
    interval="1440",
    external_axes=[ax1, ax2],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
candle_one_year = f.getvalue().decode("utf-8")

## Due Diligence

In [ ]:
# market cap dominance
mcapdom_df = openbb.crypto.dd.mcapdom(symbol=ticker)

if (isinstance(mcapdom_df, pd.DataFrame) and not mcapdom_df.empty) or (
    not isinstance(mcapdom_df, pd.DataFrame) and mcapdom_df
):
    fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
    openbb.crypto.dd.mcapdom_chart(symbol=ticker, external_axes=[ax])
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    mcapdom = f.getvalue().decode("utf-8")
else:
    mcapdom_df = pd.DataFrame()

In [ ]:
# roadmap
roadmap_model = openbb.crypto.dd.rm(symbol=ticker)

if (isinstance(roadmap_model, pd.DataFrame) and not roadmap_model.empty) or (
    not isinstance(roadmap_model, pd.DataFrame) and roadmap_model
):
    roadmap_model = roadmap_model.set_index("Date")
    roadmap_model.sort_index()

    fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
    openbb.crypto.dd.rm_chart(symbol=ticker, external_axes=[ax])
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    roadmap_view = f.getvalue().decode("utf-8")

else:
    roadmap_model = pd.DataFrame()

In [ ]:
tokenomics_model = openbb.crypto.dd.rm(symbol=ticker)

if (isinstance(tokenomics_model, pd.DataFrame) and not tokenomics_model.empty) or (
    not isinstance(tokenomics_model, pd.DataFrame) and tokenomics_model
):
    tokenomics_model = tokenomics_model.set_index("Date")
    tokenomics_model.sort_index()

    fig, ax1 = plt.subplots(figsize=(11, 5), dpi=150)
    ax2 = ax1.twinx()
    openbb.crypto.dd.tk_chart(symbol=ticker, external_axes=[ax1, ax2])
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    tokenomics_view = f.getvalue().decode("utf-8")

else:
    tokenomics_model = pd.DataFrame()

In [ ]:
# fundraise
fundraise = openbb.crypto.dd.fr(symbol=ticker)  # this returns a tuple

if fundraise:
    fundraise_info = fundraise[0]
    fundraise_df = fundraise[1]

In [ ]:
trading_pair_info = openbb.crypto.dd.trading_pair_info(f"{ticker}-USDT")
if (isinstance(trading_pair_info, pd.DataFrame) and not trading_pair_info.empty) or (
    not isinstance(trading_pair_info, pd.DataFrame) and trading_pair_info
):
    trading_pair_info = trading_pair_info.set_index("Metric")
    trading_pair_info
else:
    trading_pair_info = pd.DataFrame()

In [ ]:
import openbb_terminal.cryptocurrency.due_diligence.pycoingecko_model as gecko
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coingecko_id

ath = pd.DataFrame()
try:
    cg_id = get_coingecko_id(symbol=ticker)
    coin = gecko.Coin(ticker)
    ath = coin.get_all_time_high(currency=cg_id)

except Exception as e:
    print("Reached coingecko API limit")

In [ ]:
import openbb_terminal.cryptocurrency.due_diligence.pycoingecko_model as gecko
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coingecko_id

atl = pd.DataFrame()
try:
    cg_id = get_coingecko_id(symbol=ticker)
    coin = gecko.Coin(ticker)
    atl = coin.get_all_time_low(currency=cg_id)

except Exception as e:
    print("Reached coingecko API limit")

In [ ]:
# market stats about loaded coin

market = openbb.crypto.dd.mkt(ticker)

if (isinstance(market, pd.DataFrame) and not market.empty) or (
    not isinstance(market, pd.DataFrame) and market
):
    market = market.set_index("exchange")

    market["market_url"] = market["market_url"].apply(
        lambda x: f'<a href="{x}">{x}</a>'
    )
    market
else:
    market = pd.DataFrame()

In [ ]:
# all exchanges where loaded coin is listed
ex = openbb.crypto.dd.ex(ticker)

if (isinstance(ex, pd.DataFrame) and not ex.empty) or (
    not isinstance(ex, pd.DataFrame) and ex
):
    ex = ex.set_index("id")
    ex
else:
    ex = pd.DataFrame()

In [ ]:
# last trades
trades = openbb.crypto.dd.trades(symbol=ticker, exchange_id="gemini", to_symbol="USD")
if (isinstance(trades, pd.DataFrame) and not trades.empty) or (
    not isinstance(trades, pd.DataFrame) and trades
):
    trades = trades.set_index("Date")
    trades
else:
    trades = pd.DataFrame()

In [ ]:
# order book
order_book_raw = openbb.crypto.dd.ob(
    symbol=ticker, exchange="gemini", to_symbol="USD"
)  # this returns a dict

if order_book_raw:
    fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
    openbb.crypto.dd.ob_chart(
        symbol=ticker, exchange="gemini", to_symbol="USD", external_axes=[ax]
    )
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    order_book = f.getvalue().decode("utf-8")

In [ ]:
# price and supply related metrics for loaded coin

ps = openbb.crypto.dd.ps(ticker)
if (isinstance(ps, pd.DataFrame) and not ps.empty) or (
    not isinstance(ps, pd.DataFrame) and ps
):
    ps = ps.set_index("Metric")
    ps
else:
    ps = pd.DataFrame()

In [ ]:
# events

events = openbb.crypto.dd.events(ticker)

if (isinstance(events, pd.DataFrame) and not events.empty) or (
    not isinstance(events, pd.DataFrame) and events
):
    events = events.set_index("date")
    events["link"] = events["link"].apply(lambda x: f'<a href="{x}">{x}</a>')
else:
    events = pd.DataFrame()

In [ ]:
# social media

from openbb_terminal.cryptocurrency.due_diligence.pycoingecko_view import display_social

display_social(ticker)
fig = plt.figure()
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
social = f.getvalue().decode("utf-8")

In [ ]:
# tweets for loaded coin

tweets = openbb.crypto.dd.twitter(ticker)
tweets

In [ ]:
# github activity over time

gh_data = openbb.crypto.dd.gh(symbol=ticker)

if (isinstance(gh_data, pd.DataFrame) and not gh_data.empty) or (
    not isinstance(gh_data, pd.DataFrame) and gh_data
):
    fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
    openbb.crypto.dd.gh_chart(symbol=ticker, external_axes=[ax])
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    gh = f.getvalue().decode("utf-8")
else:
    gh_data = pd.DataFrame()

## TA


In [ ]:
ticker_data = historical_data_one_year.copy()

In [ ]:
# simple moving average
ticker_data.index.names = ["date"]

fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.ta.ma_chart(
    data=ticker_data["Close"],
    ma_type="SMA",
    symbol=ticker,
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
sma = f.getvalue().decode("utf-8")

In [ ]:
# exponential moving average
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.ta.ma_chart(
    data=ticker_data["Close"],
    ma_type="EMA",
    symbol=ticker,
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ema = f.getvalue().decode("utf-8")

In [ ]:
# zero lag exponential moving average

fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.ta.ma_chart(
    data=ticker_data["Close"],
    ma_type="ZLMA",
    symbol=ticker,
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
zlma = f.getvalue().decode("utf-8")

In [ ]:
# commodity channel index

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.ta.cci_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax1, ax2],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
cci = f.getvalue().decode("utf-8")

In [ ]:
# moving average convergence/divergence

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.ta.macd_chart(
    data=ticker_data["Adj Close"],
    symbol=ticker,
    external_axes=[ax1, ax2],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
macd = f.getvalue().decode("utf-8")

In [ ]:
# fisher transform

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 11), dpi=150)
ax3 = ax2.twinx()
openbb.ta.fisher_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax1, ax2, ax3],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
fisher = f.getvalue().decode("utf-8")

In [ ]:
# aroon indicator

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(11, 11), dpi=150)
openbb.ta.aroon_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax1, ax2, ax3],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
aroon = f.getvalue().decode("utf-8")

In [ ]:
# bollinger bands

fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.ta.bbands_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
bbands = f.getvalue().decode("utf-8")

In [ ]:
# fibonacci retracement

fig, ax1 = plt.subplots(figsize=(11, 5), dpi=150)
ax2 = ax1.twinx()
openbb.ta.fib_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=[ax1, ax2],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
fib = f.getvalue().decode("utf-8")

## QA

In [ ]:
# summary

summary = openbb.qa.summary(data=ticker_data)
summary

In [ ]:
# normality statistics and tests

# fig, _ = plt.subplots(figsize=(11, 3), dpi=150)
# openbb.qa.normality_chart(
#     data=ticker_data,
#     target="Close",
# )
# fig.tight_layout()
# f = io.BytesIO()
# fig.savefig(f, format="svg")

normality = openbb.qa.normality(
    data=ticker_data["Close"],
)
normality

In [ ]:
# box and whisker plot

fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.qa.bw(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    yearly=False,
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
bw = f.getvalue().decode("utf-8")

In [ ]:
# rolling mean and std deviation of prices

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.qa.rolling_chart(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    external_axes=[ax1, ax2],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
rolling = f.getvalue().decode("utf-8")

In [ ]:
# rolling kurtosis of distribution of prices

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.qa.kurtosis_chart(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    external_axes=[ax1, ax2],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
kurtosis = f.getvalue().decode("utf-8")

## Render the report template to a file

In [ ]:
body = ""

img = (
    str(REPOSITORY_DIRECTORY)
    + "/openbb_terminal/reports/templates/OpenBB_reports_logo.png"
)
floppy_disk_img = (
    str(REPOSITORY_DIRECTORY) + "/openbb_terminal/reports/templates/floppy-disc.png"
)
body += widgets.header(
    img,
    floppy_disk_img,
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>INVESTMENT RESEARCH REPORT:</b> {ticker}",
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Due Diligence",
        "Technical Analysis",
        "Quantitative Analysis",
    ]
)

comment = ""

# Summary tab
htmlcode = ""
htmlcode += widgets.row(
    [widgets.h(3, "Basic information about loaded coin") + basic_info.to_html()]
)
if not links.empty:
    htmlcode += widgets.row([widgets.h(3, "Links") + links.to_html(escape=False)])

if not news.empty:
    htmlcode += widgets.row(
        [widgets.h(3, "Published news") + news.to_html(escape=False)]
    )

htmlcode += widgets.row([widgets.h(3, "Historical data")])
for data in historical_data:
    htmlcode += widgets.row([data])

htmlcode += widgets.row([widgets.h(3, "1-year historical data candlestick chart")])
htmlcode += widgets.row([candle_one_year])
htmlcode += widgets.row([widgets.h(3, "Sentiment analysis")])
htmlcode += widgets.row([headlines])
# TODO: print rich table
# htmlcode += widgets.row([widgets.h(3, "Quick performance")])
# htmlcode += widgets.row([quick_performance])
body += widgets.add_tab("SUMMARY", htmlcode)

# # Due diligence tab
htmlcode = ""
if not trading_pair_info.empty:
    htmlcode += widgets.row(
        [widgets.h(3, f"Trading pair info {ticker}/USDT") + trading_pair_info.to_html()]
    )
if not mcapdom_df.empty:
    htmlcode += widgets.row([widgets.h(3, "Market cap dominance")])
    htmlcode += widgets.row([mcapdom])

if not roadmap_model.empty:
    htmlcode += widgets.row([widgets.h(3, "Roadmap") + roadmap_model.to_html()])
    htmlcode += widgets.row([roadmap_view])

if not tokenomics_model.empty:
    htmlcode += widgets.row([widgets.h(3, "Tokenomics") + tokenomics_model.to_html()])
    htmlcode += widgets.row([tokenomics_view])

if fundraise:
    htmlcode += widgets.row([widgets.h(3, "Fundraising Details")])
    htmlcode += widgets.row([widgets.p(fundraise_info)])
    htmlcode += widgets.row([fundraise_df.to_html()])

if not ath.empty:
    htmlcode += widgets.row([widgets.h(3, "All time high")])
    htmlcode += widgets.row([ath.to_html()])

if not atl.empty:
    htmlcode += widgets.row([widgets.h(3, "All time low")])
    htmlcode += widgets.row([atl.to_html()])

if not market.empty:
    htmlcode += widgets.row([widgets.h(3, "All markets")])
    htmlcode += widgets.row([market.sample(n=20).sort_index().to_html(escape=False)])

if not ex.empty:
    htmlcode += widgets.row([widgets.h(3, "Exchanges where loaded coin is listed")])
    htmlcode += widgets.row([ex.sample(n=20).sort_index().to_html()])

if not trades.empty:
    htmlcode += widgets.row([widgets.h(3, "Last trades")])
    htmlcode += widgets.row([trades.tail(20).to_html()])

if order_book_raw:
    htmlcode += widgets.row([widgets.h(3, "Order book")])
    htmlcode += widgets.row([order_book])

if not ps.empty:
    htmlcode += widgets.row(
        [widgets.h(3, "Price and supply related metrics for loaded coin")]
    )
    htmlcode += widgets.row([ps.to_html()])

if not events.empty:
    htmlcode += widgets.row([widgets.h(3, "Events")])
    htmlcode += widgets.row([events.to_html(escape=False)])

# TODO: print rich table
# htmlcode += widgets.row([widgets.h(3, "Tweets for loaded coin")])
# htmlcode += widgets.row([tweets])
# htmlcode += widgets.row([widgets.h(3, "Social media")])
# htmlcode += widgets.row([social.to_html()])

if not gh_data.empty:
    htmlcode += widgets.row([widgets.h(3, "Github activity over time")])
    htmlcode += widgets.row([gh])
else:
    htmlcode += widgets.row([widgets.h(3, "Github activity over time")])
    htmlcode += widgets.row([widgets.p("No data available.")])

body += widgets.add_tab("Due Diligence", htmlcode)

# Technical Analysis tab
htmlcode = ""
htmlcode = widgets.row([widgets.h(3, "Simple moving average")])
htmlcode += widgets.row([sma])
htmlcode += widgets.row([widgets.h(3, "Exponential moving average")])
htmlcode += widgets.row([ema])
htmlcode += widgets.row([widgets.h(3, "Zero lag exponential moving average")])
htmlcode += widgets.row([zlma])
htmlcode += widgets.row([widgets.h(3, "Commodity channel index")])
htmlcode += widgets.row([cci])
htmlcode += widgets.row([widgets.h(3, "Moving average convergence/divergence")])
htmlcode += widgets.row([macd])
htmlcode += widgets.row([widgets.h(3, "Fisher transform")])
htmlcode += widgets.row([fisher])
htmlcode += widgets.row([widgets.h(3, "Aroon indicator")])
htmlcode += widgets.row([aroon])
htmlcode += widgets.row([widgets.h(3, "Bollinger bands")])
htmlcode += widgets.row([bbands])
htmlcode += widgets.row([widgets.h(3, "Fibonacci retracement")])
htmlcode += widgets.row([fib])
body += widgets.add_tab("Technical Analysis", htmlcode)

# Quantitative Analysis tab
htmlcode = ""
htmlcode = widgets.row([widgets.h(3, "Summary") + summary.to_html()])
htmlcode += widgets.row(
    [widgets.h(3, "Normality statistics and tests") + normality.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "Box and whisker plot")])
htmlcode += widgets.row([bw])
htmlcode += widgets.row([widgets.h(3, "Rolling mean and std deviation of prices")])
htmlcode += widgets.row([rolling])
htmlcode += widgets.row([widgets.h(3, "Rolling kurtosis of distribution of prices")])
htmlcode += widgets.row([kurtosis])
body += widgets.add_tab("Quantitative Analysis", htmlcode)

body += widgets.tab_clickable_and_save_evt()
report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# # to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)